In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch

In [ ]:
processor = SpeechT5Processor.from_pretrained("../speech/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("../speech/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("../speech/speecht5_hifigan")

In [ ]:
# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset(path="../speech/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [ ]:
sample_text = """
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
"""

In [ ]:
import numpy as np
arr = np.array([], dtype=np.float64)

for line in sample_text.splitlines():
    if line.strip() == "":
        continue
    inputs = processor(text=line, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    arr = np.concatenate((arr, speech.numpy()))

In [ ]:
import IPython.display as ipd

ipd.Audio(arr, rate=16000)